In [12]:
!pip install datasets # Install the Hugging Face datasets library

from datasets import load_dataset # Import the load_dataset function

# Load the SQuAD dataset
squad_dataset = load_dataset("squad")

# Print some information about the dataset
print("Number of training examples:", len(squad_dataset["train"]))
print("Number of validation examples:", len(squad_dataset["validation"]))

# Print an example question-answer pair
print("Example question:", squad_dataset["train"][0]["question"])
print("Example answer:", squad_dataset["train"][0]["answers"])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 23.2 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Number of training examples: 87599
Number of validation examples: 10570
Example question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Example answer: {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [ ]:
!pip install transformers # Install the Hugging Face transformers library

from transformers import pipeline, AutoTokenizer # Import the pipeline and AutoTokenizer classes

# Load the SQuAD model and tokenizer
model = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model)

# Create a pipeline for question answering
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Define a function to ask a question and get an answer
def answer_question(question, context):
  answer = qa_pipeline({"question": question, "context": context})
  return answer["answer"]

# Test the function on an example question and context from the SQuAD dataset
example_question = "What is the capital city of France?"
example_context = "Paris is the capital city of France."
answer = answer_question(example_question, example_context)
print("Answer:", answer)

In [ ]:
!pip install transformers # Install the Hugging Face transformers library

from transformers import pipeline, AutoTokenizer # Import the pipeline and AutoTokenizer classes

# Load the SQuAD model and tokenizer
model = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model)

# Create a pipeline for question answering
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Ask the user for a question and context, and print the answer
while True:
  question = input("Ask a question: ")
  context = input("Enter some text to search for the answer: ")
  answer = qa_pipeline({"question": question, "context": context})
  print("Answer:", answer["answer"])

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
%cd /content/gdrive/MyDrive/NLPProject/

/content/gdrive/MyDrive/NLPProject


In [21]:
!cp /content/gdrive/MyDrive/NLPProject/evaluate-v2.0.py ./evaluate-v2.0.py
!cp /content/gdrive/MyDrive/NLPProject/prediction.json ./prediction.json

cp: '/content/gdrive/MyDrive/NLPProject/evaluate-v2.0.py' and './evaluate-v2.0.py' are the same file
cp: '/content/gdrive/MyDrive/NLPProject/prediction.json' and './prediction.json' are the same file


In [26]:
!python /content/gdrive/MyDrive/NLPProject/evaluate-v2.0.py /content/gdrive/MyDrive/NLPProject/dev-v2.0.json /content/gdrive/MyDrive/NLPProject/prediction.json

{
  "exact": 64.81091552261434,
  "f1": 67.60971132981278,
  "total": 11873,
  "HasAns_exact": 59.159919028340084,
  "HasAns_f1": 64.7655368790259,
  "HasAns_total": 5928,
  "NoAns_exact": 70.4457527333894,
  "NoAns_f1": 70.4457527333894,
  "NoAns_total": 5945
}


In [30]:
import json

def calculate_f1(prediction, ground_truth):
    prediction_tokens = prediction.split()
    ground_truth_tokens = ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def calculate_em(prediction, ground_truth):
    return int(prediction == ground_truth)

def evaluate_squad(eval_file, pred_file):
    with open(eval_file, "r", encoding="utf-8") as f:
        dataset = json.load(f)["data"]

    with open(pred_file, "r", encoding="utf-8") as f:
        predictions = json.load(f)

    f1_total = 0
    em_total = 0
    for article in dataset:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                qid = qa["id"]
                answers = qa["answers"]
                if not answers:
                    continue
                answer_texts = [a["text"] for a in answers]
                if qid not in predictions:
                    continue
                prediction = predictions[qid]
                f1 = 0
                em = 0
                for answer_text in answer_texts:
                    cur_f1 = calculate_f1(prediction, answer_text)
                    if cur_f1 > f1:
                        f1 = cur_f1
                        em = calculate_em(prediction, answer_text)
                f1_total += f1
                em_total += em

    f1_score = 100.0 * f1_total / len(predictions)
    em_score = 100.0 * em_total / len(predictions)

    print("F1 score: {:.2f}".format(f1_score))
    print("EM score: {:.2f}".format(em_score))

eval_file = "/content/gdrive/MyDrive/NLPProject/dev-v2.0.json"
pred_file = "/content/gdrive/MyDrive/NLPProject/prediction.json"
evaluate_squad(eval_file, pred_file)

F1 score: 32.02
EM score: 28.49
